# ARP Lab
## A Retirement Planning Laboratory

This package is a retirement modeling framework for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment for exploring *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have a certain asset earnings plan. This does not mean one cannot make decisions. These decisions need to be guided with an understanding of the sensitivity of the parameters.This is exactly where this tool fits it. Given your savings and spending desires, it can generate different future realizations of your strategy under different market assumptions, helping to better understand your financial situation.

Copyright - Martin-D. Lacasse (2023)

Disclaimers: *I am not a financial planner. You make your own decisions. This program comes with no garantee. Use at your own risk.*

### <span style="color: blue"> README FIRST </span>
<span style="color:black;background:yellow;font-weight:bold">
Don't make changes directly to this file. Keep it as a working example. Therefore, it is recommended that you build your own case using the template files provided where you will be able to enter your own numbers and explore your own assumptions.
</span>

# The case of Jack and Jill

## Introduction
This file is provided as an example to introduce you to ARP Lab.

It is assumed that you have some familiarity with using a jupyter notebook or jupyterLab, and some basic programming skills in Python. If not, a simple tutorial can guide you to the basic skills needed.

For simulating your own realizations, use the files beginning with *template*. Make a copy and rename them keeping the same extension and give them you own names. Then you'll be able to personalize a case with your own numbers and start experimenting with ARP Lab.

This notebook named *jack+jill* describes the case of Jack and Jill, a ficticious couple used for demonstration.

### Just some Python module bookkeeping
The following commands are bookkeeping calls that load the required module and set the program to display verbose informational messages as it runs. It also makes sure that all graphs are properly displayed within the jupyter interface.

These commands need to be at the beginning of an ARP Lab notebook.

In [ ]:
%matplotlib inline
# import arp

import importlib
arp = importlib.import_module('arp')
importlib.reload(arp)

arp.setVerbose(True)

## Initializing the realization
In order to be able to generate a realization of the future, one must start with providing the year of birth of each spouse(s) and their expected lifespan(s).

For selecting your own numbers, there are plenty of longivity predictors on the web. Pick your favorite:

https://longivityillustrator.org

https://www.livingto100.com/calculator

https://www.sunlife.ca/en/tools-and-resources/tools-and-calculators/life-expectancy-calculator/

or just Google life expectancy calculator.

There are two values needed for couples. Single individuals just enter one value in each list between square brackets `[ ]`. For couples, always keep the same order in the pair of values when entering the data.

When first creating a plan, the default values used for assets allocation ratios and rates of return will be reported. These values are listed to make you aware that these values would be used if no other choice is entered. We will cover how to do this in the next sections.

Here Jack was born in 1961 and Jill in 1964. Jack hopes to live to 89 years old, while Jill thinks she might reach age 92.

In [ ]:
plan = arp.Plan(YOB=[1961, 1964], expectancy=[89, 92])

## Specify account balances and spousal beneficiaries
For each spouse, savings accounts have three buckets comprising of the total value of:
- Individual **taxable** investment or savings accounts, including bank accounts, and CDs - do not include your safety net account which should typically be sufficient for sustaining 6 months of living expenses;
- **Tax-deferred** savings accounts, including all IRAs, 401k, 403b, etc.;
- **Tax-free** savings accounts, including Roth IRAs and Roth 401k.

For married couples, each spouse will have to enter values for each type of savings account, following the same order as before. For single individuals, only one value is needed between square brackets `[ ]`.

Most investment accounts have named beneficiaries. The *beneficiary* values specify the fraction of total of assets left to the other spouse at death. For example, a spouse leaving 3/4 of her fortune to her three children and the other part to her partner would have a beneficiary value of 0.25. While this number is irrelevant for single individuals, it still needs to be entered: just use `[1]`.

Jack and Jill leave everything to each other as savings account beneficiaries as entered by [1, 1]. Jack has \\$90k in his taxable account, \\$600k in his 401k, and $60k in his Roth 401k and Roth IRAs. Jill has \\$60k in her savings bank account, \\$150k in a 403b, and \\$40k in Roth IRAs in which she contributed over the years.

In [ ]:
plan.setAssetBalances(taxable=[90000, 60000], 
                      taxDeferred=[600000, 150000],
                      taxFree=[60000, 40000],
                      beneficiary=[1, 1])

## What are the current and future assets allocations?
Each savings account can invest in 4 major classes of assets:
- Equity funds tracking the S&P 500 index;
- Bond assets tracking the Corporate bonds (Baa) index;
- Fixed-income securities represented by the performance of 10-year Treasury notes;
- Inflation-indexed securities tracking the urban Consumer Price Index (common assets).

The total of percentages in each class of assets for each savings account must add to 100%.

You are asked to provide assets allocation ratios for today, and ones for the time at the end of your life.
Values in between will be interpolated using a linear operator (for now). This can be useful
if you want to shift assets allocation as you age.


In the example below, Jack's and Jill's assets allocations start in full stock equities in the tax-free account,
gradually transitioning to a more conservative portfolio towards the end of their life. Jill's tax-deferred
account stays as a traditional 60/40 for the duration of the realization. Note that assets allocation ratios are entered as percentages, and that these percentages are for each type of savings account, and for each spouse. This strategy does not account for the balances in each account, but ARP Lab can deal this complexity as we will see in the advanced allocation strategies.

In [ ]:
plan.setInitialAR(taxable=[[0, 25, 50, 25], [0, 25, 50, 25]],
                  taxDeferred=[[50, 50, 0, 0], [60, 40, 0, 0]],
                  taxFree=[[100, 0, 0, 0], [100, 0, 0, 0]])

plan.setFinalAR(taxable=[[0, 25, 50, 25], [0, 25, 50, 25]],
                taxDeferred=[[30, 50, 10, 10], [60, 40, 0, 0]],
                taxFree=[[30, 50, 10, 10], [60, 40, 0, 0]])

plan.interpolateAR('linear')

#### Advanced assets allocation strategies
When savings account balances differ significantly between accounts, it might be beneficial to push one type of asset into a specific account. For example, stocks are more likely to grow in the long-term and therefore are better suited for tax-free accounts, while secure and less-performant assets are better held in taxable accounts where growth would be taxed. Therefore, one can maintain a balanced overall portfolio while shifting assets differently in specific savings accounts.

In some other cases, it might be even more beneficial to coordinate both spousal accounts in order to optimize assets allocations. This is the case for example when only one spouse has a tax-free account. For these situations, we can instruct ARP Lab to coordinate assets allocations between spousal accounts.

These advanced assets allocation strategies are done using the `setCoordinatedAR()` function call. There are two choices: one can provide a list of double entries, in which case the assets allocations for each spouse will be done separately. Here is one example:

    plan.setCoordinatedAR(initial=[[60, 30, 5, 5], [70, 30, 0, 0]],
                          final=[[50, 30, 5, 15], [40, 20, 15, 25]])
    
where each spouse chose different allocation ratios. In this case, assets allocations will only be coordinated between accounts belonging to the same individual.

This is in contrast with the next example, where accounts of both spouses will be coordinated together. This is done by providing a single set of ratios for each bound (*initial* and *final*):

    plan.setCoordinatedAR(initial=[60, 30, 5, 5], final=[50, 30, 5, 15])
    
With these options, ARP Lab will try to pack as much stocks into the tax-free accounts, then into the tax-deferred accounts, then into the taxable accounts. For common assets, the reverse order is used and ARP Lab will attempt packing as much common assets in the taxable accounts, then the tax-deferred, etc. In these cases, the allocation ratios depend on the balance in each of the accounts, but the overall ratios of all savings accounts under consideration will be matching to one prescribed by the `setCoordinatedAR()` function. As the account balances will be evolving from one year to the other, so will the calculated ratios. At the end of a scenario, the calculated ratios can be saved in an Excel spreadsheet by using the `saveInstance()` function that will be introduced below.

As for the non-coordinated case, a call to `setCoordinatedAR()` must be followed by an interpolation as before. Examples of such commands are in the next cell, and are commented out. Just delete the `#` before the commands to enable the statements.
    

In [ ]:
# For coordination taking place for each individual separately.
# plan.setCoordinatedAR(initial=[[60, 30, 5, 5], [70, 30, 0, 0]], final=[[50, 30, 5, 15], [40, 20, 15, 25]])

# For a single individual, of for coordinating both spousal accounts simultaneously.
# plan.setCoordinatedAR(initial=[60, 30, 5, 5], final=[50, 30, 5, 15])

# An interpolation must follow either call for generating the data for all the years of the simulation.
# plan.interpolateAR('linear')

## What about anticipated fixed income?
Pension and social security are fixed income. Model here assumes that pension income is not inflation adjusted while social security benefits are (but ARP Lab can easily be modified to account for inflation-adjusted pensions). Numbers to be provided are the predicted amount for each spouse and the age of the commencement of benefits.

By default, no pension benefits are assumed. This can also be specified explicitly by entering zeros (0) as entries, as in

    plan.setPension([0, 0], [65, 65])
    
For social security, one must provide the predicted amount(s) and the starting age(s) at which benefits are anticipated to be received. There are plenty of social security benefit estimators on the web, including the info you can get directly from your own account at the Social Security Administration (ssa.gov). Another interesing calculator can be found at www.opensocialsecurity.com. This calculator allows you to compare different scenarios regarding your commencement age through a sensitivity plot.


Here, Jill has an unindexed pension of \\$10k per year. Both Jack and Jill believe they have good genes and decided to take their social security benefits at age 70. The amounts provided are estimation of the amounts they would receive at age 70. 

In [ ]:
plan.setPension([0, 10000], [65, 65])

plan.setSocialSecurity([28000, 25000], [70, 70])

## There must be a plan for savings
The most controllable part of retirement planning is the control one has over work income, contributions to savings accounts, Roth conversions, and other big spending items in the near- and mid-term future.
In order to execute a realization, one must provide an earning, saving, and Roth conversion plan. This is done through providing an Excel workbook with one spreadsheet (tab) per spouse with the following information:

|year|anticipated income|ctrb taxable | ctrb 401k | ctrb Roth 401k | ctrb IRA | ctrb Roth IRA | Roth X | big ticket items|
|--|--|--|--|--|--|--|--|--|
|2023 | | | | | | | | |
|2024 | | | | | | | | |
| ... | | | | | | | | |
|20XX | | | | | | | | |

Here, 20XX is the last row that is the first year after the last spouse has passed based on the life expectancy values provided. For the columns, *anticipated income* is the annual amount (gross) that you anticipate to receive from employement or other sources (not including dividends from your taxable investment account). Note that column names are case sensitive and all of these entries must be in lower case. Best way to start this process is to use the template provided rightly named *template.xlsx*.

For the purpose of this exercise, there is no clear definition of retirement age. There will be a year, however, from which you will stop having anticipated income, or diminished income due to decreasing your work load. This transition can be gradual or sudden. Therefore there is no need to enter a retirement age for the sole purpose of quantifying your financial future.

Contributions to your savings accounts are marked as *ctrb*. Contributions to your 401k must also include your employer's contributions. As this file is in Excel, one can use the native calculator to enter a percentage of the anticipated income for contributions as this can sometimes be easier.

Roth conversion are specified in the column marked *Roth X*. Roth conversion are typically performed in the years when the income is lower (and therefore lower tax rates), typically in the bridge years between having a full-time regular salary and collecting social security.

Finally, *big ticket items* are used for accounting for the sale or purchase of a house, or any other major expense or money that you would give or receive (e.g., inheritance, or large gifts to or from you). Therefore, the sign (+/-) of entries in this column is important. All other column entries should be positive.

The tab name for each spreadsheet represents the name of the spouse for reporting yearly transactions affecting the plan. In fact, ARP Lab extracts the names of these tabs to determine the individuals' names. Therefore, you need to rename these tabs to reflect your personal names if you want to accurately represent your case.

Note that the file format from the (free) LibreOffice software can also be read, so you do not need to have an Excel license.

Jack and Jill has provided their specific information in the file *jack+jill.xlsx*. This worksheet file needs to have two tabs, the first one named *Jack* and the second one named *Jill*, following the same order of the data provided before. Open this file in Excel and familiarize yourself with its contents.

In [ ]:
plan.readContributions('jack+jill.xlsx')

## How much net income is desirable at retirement?
For determining the desirable annual net income in retirement, certified planners will strongly suggest that you've must have already done a cashflow analysis on your yearly spending. After this exercise, you should have a good idea of how much you'll need in retirement. Another approach is to experiment with multiple spending scenarios and see what your current and future savings can sustain under different market conditions. As one nears retirement, both approaches will need to meet at one point.

The desired income defined here is the minimum annual **net** income (i.e., after paying federal income tax) that one would like to have starting at her/his "retirement age" (we will provide a loose definition of the term *retirement age* below). This desired income must be adjusted for inflation and can follow an additional adjustment called a *smile* profile. A *smile* profile accounts for the fact that your spending capacity will modulate during retirement as you go from the so-called gogo years to the no-go years. A *flat* profile, on the other hand, will keep the same value, which will only be adjusted for inflation.

The target and actual net income values achieved by the realization can be plotted as will see below.


Jack and Jill believe that their spending profile will follow a *smile* curve rather than a *flat* line. They also believe that they can be comfortable with \\$7k per month, therefore \\$84k per year.   Can they afford it?

In [ ]:
plan.setDesiredIncome(84000, 'smile')

## How to withdraw from spousal accounts?
It can be desirable for couples to determine how to make distributions from each spousal account.
By specifying an *auto* spousal split, the withdrawals from the retirement savings accounts will be made proportial to their respective balances. It can also be specified as a fixed fractional value (e.g., 0.65) in which case 0.65 will be taken from the first account and the other 35% taken from the other (second entry) spouse, at this risk of depleting one account before the other, and running short on income.

All withdrawals use a smart banking approach which favors depleting taxable accounts before tax-deferred, before tax-free. With *auto*, additional checks and bounds are used to better coordinate spousal accounts.


Jack and Jill decide to rely on ARP Lab to make a proportional withdrawal decision on their behalf.

In [ ]:
plan.setSpousalSplit('auto')

## Specify rates of return and inflation rate
Rates of return for each class of assets can be specified, including the rate of inflation.
These are important components for  future assumptions.
Setting future rates is done with the `setRates()` method from which one can select different
sources for determining the ratesfor.
Valid choices are *historical*, *stochastic*, *average*, 'fixed', or 'default'.

For the *historical*, *stochastic*, and *average* options, data from 1928 to the last year are available for experimenting.
Ranges chosen smaller than the life horizon of the longest-lived individual will have rate values repeated in cycle. For example,
choosing historical data from 1994 to (up and including) 1996 will repeat these three values over the time span of the realization.
This would be called as follows:

    plan.setRates('historical', 1994, 1996)

This case is only provided as an explanatory example, as it would have little practical value.

If the upper bound is not provided as the third argument, then the latest data year (i.e., last year) will be assumed by default.
If one chooses a historical range starting from 1970, ARP Lab will use the rates of 1970 for this year and 1971 for next, etc. This would be simulated as follows:

    plan.setRates('historical', 1970)
  
Due to its particular sequence of rates, the worst-case historical scenario is a retirement starting in 1966. This can be simulated as follows:

    plan.setRates('historical', 1966)

In this case, the  current year will have the same rates as those that happened in 1966, and next year will have those from 1967, and so on. This choice is given for instructional purposes only. No one should make a plan based on the worst-case historical scenario. Nevertheless, it can be informative to test your own case. In practice, however, a success rate larger than 90% over a reasonable set of historical starting years and market assumptions would be acceptable by a large portion of rational thinkers. But this is all a personal choice. We'll cover how to model multiple starting years below.

Alternatively, one can choose a *stochastic* approach in which case the rates are determined from the multivariate distribution for the 4 rates in the selected year range.  The computed statistical distribution of the selected range of data is used to generate random rate values. For example,

    plan.setRates('stochastic', 1945)
    
will analyze the annual rates from 1945 up to last-year and compute means and covariance to generate new data that are statistically representative of the ones observed during this selected time period. The rates randomly generated for the timespan can be plotted and examined as we will see below. Similarly,

    plan.setRates('stochastic', 1940, 1970)

would generate random rates consistent to those observed during the 1940 - 1970 time period.

Rates can also be set to fixed values obtained from an average over a time interval using the *average* option. For example, the call

    plan.setRates('average', 1990, 2020)

would set the rates to fixed values being the average observed from 1990 to 2020 inclusively.
    
Finally one can also use fixed annual rates by providing a list of 4 entries in percent as follows:

    myrates = [9.6, 4.0, 3.0, 3.8]
    plan.setRates('fixed', values=myrates)
    
This example would use fixed rates of 9.6%, 4.0%, 3.0%, and 3.8% as average annual returns on S&P500, corporate bonds, Treasury notes, and common assets, respectively, with an average annual inflation rate of 3.8% for the full duration of the time simulation. Recall that the common assets class consists of investments tracking inflation only. Therefore the last index serves both to track the common asset class and to adjust values for inflation.

Also note that the S&P500 rates provided always include dividends, which are assumed to be reinvested.


Jill is interested to know if her situation would survive a retirement started in 1969. The following call would model exactly that situation.

In [ ]:
plan.setRates('historical', 1969)

Since ARP Lab has the historical rates available, one can also display their histograms using a simple function call. This is done with `arp.showRateDistribution()`.

Given the standard deviation of each histogram, the risk/benefit between stocks and bonds is clear. As this function returns the means and covariance matrix, we assign the result to the `_` variable which, in Python, is used to store results from the last command.

In [ ]:
_ = arp.showRateDistributions(1928, 2022)

## Generating the outcome of a scenario
We're now ready to run a single instance of a scenario. 

Recall that we set the rates above to mimic those from and after 1969. For this purpose, we only need to run a single realization looking at what would happen if Jack and Jill had retired in 1969. So let's run this plan: This following call runs all the required calculations for the time horizon over Jack and Jill's life expectancies. `run()` returns many variables that we assign to the `_` variable, used to store the answer to the last command in Python. Just a trick to avoid printing all these numbers on screen. Having set the variable to verbose, `run()` will report what happens during each year modeled.

In [ ]:
_ = plan.run()

# Analysis

### Show net income compared to target income over the years
This graph shows how the actual net income generated by the plan realization matches the inflation-adjusted net income profile specified. Note the 40% drop in target income as one spouse passes. This drop is configurable through the `setSurvivorFraction()` method. Default value is 60%.

In [ ]:
plan.showNetIncome();

### Show sources of income over the years
Income will typically come from mutiple sources, and it can be quite complex. This graphs shows the breakdown of Jack and Jill's sources of income by spouse and by origin. Note that distributions from tax-deferred accounts (*dist*) are distinguished from required minimum distributions (*rmd*) as they serve different purposes. Other labels should be self-explanatory.

In [ ]:
plan.showSources();

### Show savings accounts balances at the beginning of each year
The balance for each savings account for each spouse is calculated at the beginning of each year. Another important aspect of this graph is how much is left at the end of the realization. This will be addressed in the next cell.

In [ ]:
plan.showAccounts();

To get more information about accounts at the last year of the realization, one can use

    plan.setDeferredTaxRate(30)
    plan.estate()
    
which returns the value of the assets in today's \\$ at the last year of the scenario, assuming (in this case) a 30\% tax burden on the taxable portion of the estate (read tax-deferred savings accounts). The `estate()` function returns two values: the total post-tax value of all savings account in today's dollars and the cumulative inflation rate between today and the last day of the realization. We can also override the tax-deferred rate value by giving an argument. In the case below, a value of 35% would override the value shown above.

In [ ]:
plan.estate(35)

### Show assets allocations during the time period
The allocation of assets is shown for the three types of savings accounts (taxable, tax-deferred, and tax-free) for each spouse and for the 4 types of investments: stocks (S&P 500), corporate bonds (Baa), Treasury notes (10-y), and common assets tracking inflation.

In [ ]:
plan.showAssetsAllocations()

### Show annual taxes paid over the years of the realization
This graph shows how much Jack and Jill paid in federal taxes and IRMAA income-related Medicare insurance monthly adjustments over the years of this realization.

In [ ]:
plan.showTaxes();

### Show taxable annual income and anticipated tax brackets
Gross income also includes Roth conversions and big-ticket items, both of which are not contributing to your net income. This graph shows Jack and Jill's gross taxable income and how it compare with some anticipated Federal tax marginal brackets. This visualization is very convenient when one wants to perform Roth conversions and remain below a certain tax bracket.

Note the shift in tax brackets taking place as the Tax Cut and Job Act expires after 2025.

In [ ]:
plan.showGrossIncome();

### Show annual rates used for calculations
As described above, there are many choices for selecting rates. This graph will display the annual rates used during the time span of this realization.

In [ ]:
plan.showRates();

## Can we save this realization?
This instance of a future realization contains information on the distribution amounts, including the required minimum distribution that had to be performed under the given assumptions. This info can be saved in an excel workbook with one spreadsheet (tab) for each spouse. Worksheet will also contain annual rates, income, income taxes, and account balances. Second `True/False` argument controls if existing files get overwritten. Also remember that Windows will not allow the file to be overwriten while the file is being opened in Excel. In that case, the script will ask you to close the file and will retry to save.

Here, this call will create an excel workbook with one spreadsheet (tab) for Jack and one for Jill. 

Open the file in Excel to see what it looks like. 

In [ ]:
plan.saveInstance('Jack and Jill - 1969', True)

## Next level
ARP Lab allows one to run multiple simulations in one call. For example, one can run multiple historical cases over a range of years. In those simulations, multiple instances are executed to further explore the robustness of one's decisions under different market realizations. In another case, one can run a Monte-Carlo simulation in which ficticious rates derived from the statistical distribution of those observed in a selected time period are used for the time span of the simulation.


In these simulations, multiple scenarios will be generated and need to be compared. For ranking outcomes, it is instructive to compare the final value of the estate, if any. Given that parts of the assets portfolio are taxable, one has to provide an estimate of the income tax rate to be applied to the taxable portion of the final assets. This is done using the `setDeferredTaxRate()` method, by which an estimated tax rate is applied on the tax-deferred portion of the estate for comparison in the last year of the scenario. The default value is 25%. Note that this rate is the marginal tax rate that will be paid by the heirs, not including possible additional estate taxes.

In [ ]:
plan.setDeferredTaxRate(30)

### Historical comparisons
The `runHistorical()` function runs multiple scenarios each going through a historical sequence of rates starting at a given year. It simulates a situation in which this year's rates would be those which happened. This scenario simulates future rates based on a past sequence. As such, the next simulation year would take the rates which happened in 1961 and so on.

User can select what values to plot at the end of each scenario through the `myplots` keyword. It understands the following words: *net income*, *rates*, *taxable income*, *sources*, *taxes*, *accounts*, and *allocations*.

We will run Jack and Jill's scenario over many years. Notice how their scenario is successful for all sequences except for those starting in 1966 if they are willing to spend \\$84,500 or less as inflation-adjusted net income. Also notice the difference in net income and final estate between the sequences starting in 1974 and in 1975. It is surprising how much difference can one year of delay make in this case.

In [ ]:
plan.setDesiredIncome(84500, 'smile')
arp.setVerbose(False)
plan.runHistorical(1960, 1988, myplots=['rates', 'net income'])

### Monte-Carlo simulations
In a Monte-Carlo simulation, a random number generator is used to create values that meet the statistical criterion of a known distribution. In ARP Lab, one specifies the range of years of historical annual rates data that will be used to characterize the probability distribution of a normal multivariate distribution. Once selected, one can run as many realizations as desired. The more is generally the better, as it will improve the accuracy of the mean value while increasing the possibility of observing rare events. A value of 1,000 is generally more than sufficient. For the sake of demonstration in this notebook, we will use a smaller number of iterations of 500.

In [ ]:
plan.runMonteCarlo(500, 1960, 2022)

## One more notch up for modeling
We can also run Roth conversion estate maximization strategies. Using commutativity of multiplication, one can easily demonstrate that there is no point in doing a Roth conversion if the tax rate to be paid on the conversion now is the same as the one to be paid on the distribution later. However, things are much more complex in reality, and one has to take RMDs in consideration on top of all other unknown factors that can happen, including changes in the tax code. For this purpose, a simple heuristical Roth optimizer is being developed. For now, the objective function is to maximize the estate amount left at the passing of the last spouse, considering a tax bracket on the taxable portion of the estate, and under the constraint of receiving a desired income. Function will return a new plan, and an array containing proposed Roth conversions. These conversions are not yet fully optimal, as the current algorithm is biased towards early conversions.

Here is an example of how it works: 

In [ ]:
plan2, conversions = arp.optimizeRoth(plan, 30)
print(conversions)
plan2.run();
plan2.saveInstance('plan2', True)

The Excel file named *instance_plan2.xlsx* saved will contain all the information for the heuristically optimized plan. Take a few minutes to open this file and explore the different tabs.

In this case, Roth conversions increased estate by $241k without compromising the desired income. The difference lies in RMDs that push the gross income into higher bracket, and the fact that the tax-free account had a different assets allocation. At this point, both plans can be compared using these lines of code:

In [ ]:
plan.showNetIncome()
plan2.showNetIncome()
plan.showSources()
plan2.showSources()
plan.showAccounts()
plan2.showAccounts()

### Saving configuration parameters
Configuration parameters set for a run can be saved in a file, thus avoiding to re-enter the same information in the future. This is done using the `savePlan()` and `readPlan()` function calls. Note that 'readPlan()' returns a newly created plan pre-configured according to the parameters set in the configuration file.

In [ ]:
# To save all configuration parameters
arp.savePlan(plan, 'jack+jill')
             
# To create a new plan configured according to the parameters in configuration file.
newplan = arp.readPlan('jack+jill')

This notebook is provided as a basic example on what you can do to assess the robustness and sensitivity of your retirement strategy. Go and explore!


Enjoy!
